# System Updates: Adding LLM

Charts: https://www.canva.com/design/DAGAzyZZsoY/rkmJ2lIRU_38ONCR0-rifQ/edit?utm_content=DAGAzyZZsoY&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton

Basically we need to figure out how to include LLM into the system. Based on the chart above, the input will be some form of the student's challenge. Optional additions would be *other* students similar challenges and/or their solutions (to be added to the LLM prompt). We will experiment with what prompts to use for good outputs.   

## Step 1. Connect to LLM (ChatGpt) API

In [28]:
# First: Install it
!pip install openai==0.28.1

  Using cached openai-0.28.1-py3-none-any.whl.metadata (11 kB)
Using cached openai-0.28.1-py3-none-any.whl (76 kB)


In [32]:
# debugging stuff/some steps while figuring it out
#!pip uninstall typing_extensions --yes
#!pip install typing_extensions==4.7.1
#!pip install typing_extensions==4.7.1 --upgrade
#!pip install typing_extensions==4.8 --upgrade

In [18]:
import openai
import json
import time

We have imported an older version so things don't break (:

First, get the API key so we can actually do stuff. (:

In [19]:
# Load the configuration from the JSON file
with open("../configs/openai_key.json", "r") as config_file:
    config = json.load(config_file)

In [20]:
# Access the values
openai.api_key = config["api_key"]

Some magical method from Sandra who has blessed us with this code.  
Parameters:  
- prompt: literally the prompt you are sending into chatgpt
- input: basically whatever value(s) we want the prompt to act upon

In [21]:
models = ["gpt-3.5-turbo", "gpt-4-0125-preview"]

In [22]:
def general_prompt(prompt, data, model_select=models[0]):
        if model_select == models[0]:
            request_timeout = 10
        elif model_select == models[1]:
            request_timeout = 30
        retries = 3
        for _ in range(retries + 1):  # Include the initial attempt plus the specified retries
            try:
                response = openai.ChatCompletion.create(
                    model=model_select,
                    messages=[
                        # Input the prompt
                        {"role": "system", "content": prompt},
                        # Input reflection response
                        {"role": "user", "content": data},
                    ],
                    temperature=0.7,
                    max_tokens=2000,
                    request_timeout=request_timeout,
                    
                )
                # If the request was successful, break out of the loop
                break
            except openai.error.OpenAIError as e:
                # Handle the error (e.g., print the error message)
                print("Error:", e)
            except TimeoutError:
                # Handle a timeout (e.g., print a message or take other actions)
                print("Request timed out")
            
            # Add a delay before retrying (to avoid immediate retries)
            time.sleep(1)

        if response and len(response.choices) > 0:
            prompt_response =  str(response['choices'][0]['message']['content'])
            print(prompt_response )
            return (prompt_response)

## Trying out prompts

First, we are asking it to generate advice based on the students challenge alone:

One student's challenge is: **I would say my biggest challenge was time management. It was a little difficult for me to plan out everything the way I wanted and along with my other class work that did not help with me getting the time to do everything I needed to.**

In [51]:
prompt = ''' You are an instructor helping students with their challenges. Based on this student's challenge, what advice would you send to them?'''
data = '''I would say my biggest challenge was time management. It was a little difficult for me to plan out everything the way I wanted and along with my other class work that did not help with me getting the time to do everything I needed to.'''

In [52]:
general_prompt(prompt, data)

Time management can be a common challenge for many students, but there are strategies you can use to improve in this area. Here are some tips that may help you better manage your time:

1. **Create a Schedule:** Start by creating a daily or weekly schedule that includes all your classes, assignments, study time, and other commitments. This will help you visualize how you are spending your time and identify areas where you can be more efficient.

2. **Prioritize Tasks:** Make a list of tasks and prioritize them based on deadlines and importance. Focus on completing high-priority tasks first to ensure you meet your deadlines and stay on track.

3. **Set Realistic Goals:** Be realistic about what you can accomplish in a given amount of time. Break down larger tasks into smaller, manageable steps to make them feel less overwhelming.

4. **Eliminate Distractions:** Identify and eliminate distractions that may be taking up your time unnecessarily, such as social media, TV, or unnecessary mee

"Time management can be a common challenge for many students, but there are strategies you can use to improve in this area. Here are some tips that may help you better manage your time:\n\n1. **Create a Schedule:** Start by creating a daily or weekly schedule that includes all your classes, assignments, study time, and other commitments. This will help you visualize how you are spending your time and identify areas where you can be more efficient.\n\n2. **Prioritize Tasks:** Make a list of tasks and prioritize them based on deadlines and importance. Focus on completing high-priority tasks first to ensure you meet your deadlines and stay on track.\n\n3. **Set Realistic Goals:** Be realistic about what you can accomplish in a given amount of time. Break down larger tasks into smaller, manageable steps to make them feel less overwhelming.\n\n4. **Eliminate Distractions:** Identify and eliminate distractions that may be taking up your time unnecessarily, such as social media, TV, or unnece

Some of the "similar" challenges are:  
- Time management was probably my biggest issue here this project. Seems like all of my other classes ramped up all at the same time for some reason so I felt like I rushed a bit with this project and didn't really get to do all that I wanted to do with it.
- Time management seems to be the most challenging aspect this last module, simply due to an overload of work I have to complete.
- I think the biggest challenge for me this past module was to manage my time. Because of midterm week, I found it difficult to find time all my classes and study for my midterms.Ã‚Â
- This project's biggest challenge was the due date, I had a bunch of assignments and midterms all week, so I could not focus on the project.

The current student's written solution was: "I took steps of trying to allow certain time periods to work on my work that needed to be done."  
Similar student's solutions:  
- Coffee and a very supportive partner!
- Looking ahead at assignments and prioritizing what is most important helped me sort through this past few weeks (although with work it was a struggle). It allowed me to allocate time to specific tasks that would require longer time-investment.
- I tried to priortize everthing that I could and work on the project as much as I could.Ã‚Â
- I turned in a couple days late when I could sit down and finish and give it my best effort.

In [54]:
similar_challenges = '''

Student 1:
Challenge: Time management was probably my biggest issue here this project. Seems like all of my other classes ramped up all at the same time for some reason so I felt like I rushed a bit with this project and didn't really get to do all that I wanted to do with it.
Solution: Coffee and a very supportive partner!

Student 2:
Challenge: Time management seems to be the most challenging aspect this last module, simply due to an overload of work I have to complete.
Solution: Looking ahead at assignments and prioritizing what is most important helped me sort through this past few weeks (although with work it was a struggle). It allowed me to allocate time to specific tasks that would require longer time-investment.

Student 3:
Challenge: I think the biggest challenge for me this past module was to manage my time. Because of midterm week, I found it difficult to find time all my classes and study for my midterms.Ã‚Â
Solution: I tried to priortize everthing that I could and work on the project as much as I could.Ã‚Â

Student 4:
Challenge: This project's biggest challenge was the due date, I had a bunch of assignments and midterms all week, so I could not focus on the project.
Solution:  I turned in a couple days late when I could sit down and finish and give it my best effort.

'''

In [55]:
prompt = f''' You are an instructor who wants to help students with their challenges but also increase students' sense of belonging.
You are given a current students' reflection called Student Reflection. 

The reflection prompts were What was your biggest challenge? and 'What is a potential solution to your challenge?'

You are also given former students' reflections that should have similar challenges and the solutions they suggested.

Please write an email to the student that accomplishes the following tasks:
(1) Assures the student that they are not alone with their challenge, and summarize the challenges of previous students.
(2) Summarize the previous students' solutions as well, and comment on the usefulness of the solutions.
(3) Provide your own solutions to the challenge based on your understanding of best practices in teaching.

Students' Previous Challenges:
{similar_challenges}

'''


### We are testing with 3.5 here.

In [56]:
general_prompt(prompt, data)

Subject: You're not alone in this challenge - Solutions to improve time management

Dear Student,

I hope this email finds you well. I read your reflection and want to assure you that you are not alone in facing time management challenges. Many students in the past have also struggled with similar issues. Let me summarize some of their challenges for you:

- Student 1 felt rushed with their project due to other classes ramping up simultaneously.
- Student 2 faced time management difficulties due to an overload of work.
- Student 3 found it hard to manage time during midterm week.
- Student 4 struggled to focus on the project due to multiple assignments and midterms.

In terms of solutions, previous students have shared some helpful strategies:
- Student 1 found support in coffee and a supportive partner.
- Student 2 overcame the challenge by looking ahead at assignments and prioritizing tasks.
- Student 3 prioritized tasks and worked on the project whenever possible.
- Student 4 submit

"Subject: You're not alone in this challenge - Solutions to improve time management\n\nDear Student,\n\nI hope this email finds you well. I read your reflection and want to assure you that you are not alone in facing time management challenges. Many students in the past have also struggled with similar issues. Let me summarize some of their challenges for you:\n\n- Student 1 felt rushed with their project due to other classes ramping up simultaneously.\n- Student 2 faced time management difficulties due to an overload of work.\n- Student 3 found it hard to manage time during midterm week.\n- Student 4 struggled to focus on the project due to multiple assignments and midterms.\n\nIn terms of solutions, previous students have shared some helpful strategies:\n- Student 1 found support in coffee and a supportive partner.\n- Student 2 overcame the challenge by looking ahead at assignments and prioritizing tasks.\n- Student 3 prioritized tasks and worked on the project whenever possible.\n- 

### Now we are looking at gpt 4.

In [64]:
general_prompt(prompt, data, models[1])

Error: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=30)
Error: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=30)
Subject: You're Not Alone: Strategies for Overcoming Time Management Challenges

Dear [Student's Name],

I hope this message finds you well. I wanted to reach out to you after reading your recent reflection on the challenges you've been facing, particularly around time management. It's clear that balancing your workload and finding the time to complete everything to your satisfaction has been a struggle. I want to assure you that you are not alone in this.

Many of your peers have shared similar experiences in their reflections. For example, several students have mentioned that coinciding deadlines across different subjects, midterm weeks, and the overall increase in workload significantly impacted their ability to manage their time effectively. It's a common ch

"Subject: You're Not Alone: Strategies for Overcoming Time Management Challenges\n\nDear [Student's Name],\n\nI hope this message finds you well. I wanted to reach out to you after reading your recent reflection on the challenges you've been facing, particularly around time management. It's clear that balancing your workload and finding the time to complete everything to your satisfaction has been a struggle. I want to assure you that you are not alone in this.\n\nMany of your peers have shared similar experiences in their reflections. For example, several students have mentioned that coinciding deadlines across different subjects, midterm weeks, and the overall increase in workload significantly impacted their ability to manage their time effectively. It's a common challenge, especially in rigorous academic environments.\n\nFrom their reflections, a few potential solutions emerged. One student found relief in the support of their partner and the strategic use of caffeine, highlighting

## Load previous results csv and iterate through

In [1]:
import pandas as pd

In [3]:
results = pd.read_csv('../results/2022-11-29-22-36-00_3155.csv')

In [4]:
results.head()

,email,name,reflection,index_0,cos_score_0,challenge_0,solution_0,name_0,index_1,cos_score_1,...,index_3,cos_score_3,challenge_3,solution_3,name_3,index_4,cos_score_4,challenge_4,solution_4,name_4
0,vgovindu@uncc.edu,Vivek Govindu,My biggest challenge was understanding the Inv...,1211,0.574554,I thought that all of the assignments in modul...,Plan better and do assignments early.Â,past student,1204,0.558170,...,78,0.547493,I did not really have a big challenge this pas...,"I did not have much challenges this week, but ...",past student,156,0.538889,There haven't really been any challenges for m...,Not applicable because I don't think there was...,past student
1,elozuke@uncc.edu,Ethan Lozuke,A step in the code was not working and challen...,1423,1.000000,A step in the code was not working and challen...,I talked to my group mates and found that the ...,Ethan Kalika,695,0.720188,...,1308,0.621687,My biggest challenges for the past modules ass...,I was able to overcome this challenge by searc...,Husam Suoamra,1451,0.610749,I found that the past module is somewhat chall...,I was able to overcome this challenge by under...,Husam Suoamra
2,hstingel@uncc.edu,Hunter Stingel,Time management on my part,1399,0.780023,I think it has been time management,I overcame this challenge by working with tabl...,Aj Chiara,667,0.763546,...,114,0.748190,Time Management,I've made liberal use of the altered dashboard...,past student,242,0.748190,Time management,Stop taking naps at noon every day,past student
3,alam8@uncc.edu,Andrew Lam,I think the biggest challenge for us right now...,843,0.798328,The biggest challenge thus far has been the co...,"As stated, I will put in more time learning an...",past student,1433,0.688697,...,1339,0.672367,My biggest challenge wasn't the actual coding ...,I kept trying but had to reach out to the prof...,Jackson Solis,110,0.666350,The biggest challenge for me was learning abou...,I think I need to take some extra time on the ...,past student
4,khalonki@uncc.edu,Larissa Halonkin,Catching up on work I missed during my illness,941,0.799192,Having to catch up on the work because of my c...,I've already addressed the challenge by settin...,past student,55,0.750088,...,799,0.691652,I struggled to stay on top of work this week a...,I'm going to try to plan when I do assignments...,past student,369,0.691619,I had to work a lot so I put off assignmentsÃ‚,Start the assignments earlier in the week,past student


In [ ]:
student_name = []


In [ ]:
prompt = f''' You are an instructor who wants to help students with their challenges but also increase students' sense of belonging.
You are given a current students' reflection called Student Reflection. 

The reflection prompts were What was your biggest challenge? and 'What is a potential solution to your challenge?'

You are also given former students' reflections that should have similar challenges and the solutions they suggested.

Please write an email to the student that accomplishes the following tasks:
(1) Assures the student that they are not alone with their challenge, and summarize the challenges of previous students.
(2) Summarize the previous students' solutions as well, and comment on the usefulness of the solutions.
(3) Provide your own solutions to the challenge based on your understanding of best practices in teaching.

Students' Previous Challenges:
{similar_challenges}
'''
similar_challenges = '''

Student 1:
Challenge: Time management was probably my biggest issue here this project. Seems like all of my other classes ramped up all at the same time for some reason so I felt like I rushed a bit with this project and didn't really get to do all that I wanted to do with it.
Solution: Coffee and a very supportive partner!

Student 2:
Challenge: Time management seems to be the most challenging aspect this last module, simply due to an overload of work I have to complete.
Solution: Looking ahead at assignments and prioritizing what is most important helped me sort through this past few weeks (although with work it was a struggle). It allowed me to allocate time to specific tasks that would require longer time-investment.

Student 3:
Challenge: I think the biggest challenge for me this past module was to manage my time. Because of midterm week, I found it difficult to find time all my classes and study for my midterms.Ã‚Â
Solution: I tried to priortize everthing that I could and work on the project as much as I could.Ã‚Â

Student 4:
Challenge: This project's biggest challenge was the due date, I had a bunch of assignments and midterms all week, so I could not focus on the project.
Solution:  I turned in a couple days late when I could sit down and finish and give it my best effort.

'''

In [10]:
k = 5

In [27]:
for index, student in results.iterrows():
    
    print(student)

    current_challenge = student['reflection']
    
    # Initialize an empty list to hold the dictionaries
    student_challenges = [] # note this is for previous students. top-k similar students
    
    # Iterate through previous K student challenges and solutions to build dictionary
    for i in range(k):
    
        # Assuming there's only one row in your DataFrame as per the structure you provided
        # For multiple rows, you'd need to adjust this to iterate over rows as well
        student_info = {
            "Challenge": student[f"challenge_{i}"],
            "Solution": student[f"solution_{i}"]
        }
        
        student_challenges.append(student_info)
    
    # Formatting the list into the required string format based on the challenge-solution dictionary
    similar_challenges = "\n\n".join([
        f"Student {index + 1}:\nChallenge: {entry['Challenge']}\nSolution: {entry['Solution']}"
        for index, entry in enumerate(student_challenges)
    ])
    
    # Adding triple quotes to mimic the exact output format as asked
    similar_challenges = f"'''\n{similar_challenges}\n'''"

    # Building the prompt based on the challenge-solution pairs. TODO: Make this a parameter or something
    prompt = f''' You are an instructor who wants to help students with their challenges but also increase students' sense of belonging.
    You are given a current students' reflection called Student Reflection. 
    
    The reflection prompts were 'What was your biggest challenge?' and 'What is a potential solution to your challenge?'
    
    You are also given former students' reflections. These reflections were automatically selected by our algorithm to have the most 
    similar challenges to the student out of our dataset. However, the similarity may not be clear. Make sure the student actually
    talks about a challenge before using it in the email.
    
    
    Please write an email to the student that accomplishes the following tasks:
    (1) Assures the student that they are not alone with their challenge, and provide a general summary of previous students' challenges.
    (2) Identify the general topic the former students' reflections have in common, and identify how this topic relates
    to the current student's challenge. Possible topics include: assignments, quizzes, lectures, quality of instructor, etc
    (3) Provide a general summary of the previous students' solutions as well, and comment on the usefulness of the solutions.
    (4) Provide your own solutions to the challenge based on your understanding of best practices in teaching.
    
    Students' Previous Challenges and Solutions:
    {similar_challenges}
    '''

    print(general_prompt(prompt, current_challenge, models[0]))
    break

email                                          vgovindu@uncc.edu
name                                               Vivek Govindu
reflection     My biggest challenge was understanding the Inv...
index_0                                                     1211
cos_score_0                                             0.574554
challenge_0    I thought that all of the assignments in modul...
solution_0               Plan better and do assignments early.Â 
name_0                                              past student
index_1                                                     1204
cos_score_1                                              0.55817
challenge_1    NothingÂ  really challenging about this module...
solution_1     i need to allocate more time in order to learn...
name_1                                              past student
index_2                                                     1443
cos_score_2                                             0.548937
challenge_2    Biggest ch